# Sequences and Loops in Parsl

### Importing Libraries and Configuration

We'll be using the htex configuration for Parsl. Read more [here.]( https://github.com/Parsl/parsl/blob/master/parsl/configs/htex_local.py)

In [1]:
# External Libraries
import numpy as np

# Importing Parsl and the htex configuration
import parsl
from parsl.app.app import python_app, bash_app
from parsl.providers import LocalProvider
from parsl.channels import LocalChannel

from parsl.config import Config
from parsl.executors import HighThroughputExecutor

config = Config(
    executors=[
        HighThroughputExecutor(
            label="htex_local",
            cores_per_worker=1,
            provider=LocalProvider(
                channel=LocalChannel(),
                init_blocks=1,
                max_blocks=1,
            ),
        )
    ],
)

parsl.load(config)

### For loops in Sequence

Since sequences are executed chronologically, their steps can't be executed in parallel. For example, if f(x) is followed by g(x) which is followed by h(x), then we can't execute them in parallel since their inputs depend on the previous functions results. But we can execute multiple sequences at the same time.

For example, Let x be an input that has to be processed by f(x), g(x) and h(x). Then we can execute parallel sequences for 100 values of x as shown below. Note: f(x), g(x) and h(x) are all randomly selected functions can be changed as per the need of the experiment

<img src="parallel_sequences.png"
     style="float: left; size: 15px;" />

In [2]:
## Building the Python Apps for f(x), g(x) and h(x)

@python_app
def f(x):
    return (x**2 - 4)

@python_app
def g(x):
    import numpy as np
    return (np.sin(x) + np.cos(x))

@python_app
def h(x):
    import numpy as np
    return (np.exp(x))

In [3]:
x = np.linspace(1,100,100) 

In [ ]:
out_f = []
out_g = []
out_h = []

for element in x:
    out_f.append(f(element))

for element2 in out_f:
    out_g.append(g(element2))

for element3 in out_g:
    out_h.append(h(element3))
    
out_h = [i.result() for i in out_h]

An important thing to note in the above example is that we didn't have to evaluate the results of f(x) or g(x) before h(x) using the .result() function. We simply parsed the AppFuture objects into the next python app without evaluation. Thus, it reduces the number of separate evaluations conducted and pushes everything to be conducted in parallel at the end.

In [ ]:
print(len(out_f))
print(len(out_g))
print(len(out_h))

### Nested Loops

A nested loop is a loop within a loop. Here, one loop gives us a result which becomes the input of another loop. For example, The function f(x) generates a list of the 20 natural numbers after the given input 'x' and g(x) generates a list of a given number's square root

<img src="nested_looping.png"
     style="float: left; size: 15px;" />

In [ ]:
x = np.linspace(1,50,50)

@python_app
def natural_numbers(x):
    import numpy as np
    return np.linspace(x+1, (19+x),20)

@python_app
def square_roots(x):
    return [+x**0.5,-x**0.5]

In [ ]:
final_array = []

for element in x:
    output_list = natural_numbers(element)

    for element2 in output_list.result():
        final_array.append(square_roots(element2))
        
final_array = [i.result() for i in final_array]

In [ ]:
print(len(final_array))

### Synchronization

Synchronization is the merge of existing parallel threads to compute one final result. For example, 100 different natural numbers are first squared and then their average is evaluated

<img src="synchronization.png"
     style="float: left; size: 10px;" />

In [ ]:
@python_app
def square(x):
    return x**2

@python_app
def average(inputs=[]):
    return sum(inputs)/len(inputs)
    
output_list = []
for i in range(1,5):
    output_list.append(square(i))

mean = average(inputs=output_list).result()

In [ ]:
print(mean)

### Exclusive Routing

Exclusive Routing is the idea of parsing a result to a parallel branch if it meets a given condition

<img src="exclusive_routing.png"
     style="float: left; size: 15px;" />

In [ ]:
inputs = np.linspace(1,100,100)

def check(x):
    if x%3 == 0 and x%5 == 0:
        return True
    else:
        return False

@python_app
def square_root(x):
    return x**0.5

@python_app
def squared(x):
    return x**2

final_list = []

for x in inputs:
    if check(x):
        final_list.append(square_root(x))
    else:
        final_list.append(squared(x))

In [ ]:
final_list = [i.result() for i in final_list]

In [ ]:
print(len(final_list))